In [1]:
from math import exp
from random import seed
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
    return (2/(1+exp(-2*activation))) - 1

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return 1 - (output*output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append((expected[j] - neuron['output']))
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[0]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


In [2]:
# Test training backprop algorithm
seed(1)
dataset = [[0,2.7810836,2.550537003],
	[0,1.465489372,2.362125076],
	[0,3.396561688,4.400293529],
	[0,1.38807019,1.850220317],
	[0,3.06407232,3.005305973],
	[1,7.627531214,2.759262235],
	[1,5.332441248,2.088626775],
	[1,6.922596716,1.77106367],
	[1,8.675418651,-0.242068655],
	[1,7.673756466,3.508563011]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 10, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=20.001
>epoch=1, lrate=0.500, error=6.009
>epoch=2, lrate=0.500, error=7.245
>epoch=3, lrate=0.500, error=7.490
>epoch=4, lrate=0.500, error=7.579
>epoch=5, lrate=0.500, error=7.604
>epoch=6, lrate=0.500, error=7.609
>epoch=7, lrate=0.500, error=7.607
>epoch=8, lrate=0.500, error=7.603
>epoch=9, lrate=0.500, error=7.598
[{'weights': [0.13419252906881313, 0.6923368171082648, 0.6685093822576059], 'output': 0.9999902467820663, 'delta': -5.765372818544188e-08}, {'weights': [0.46240374751192537, 1.4102818394645675, 0.3821172715309973], 'output': 0.9999999998529447, 'delta': -1.632755527499333e-11}]
[{'weights': [0.021533153359603194, 0.40568379607469407, -0.48714155635881334], 'output': 0.12745797849574192, 'delta': -0.1253873552816308}, {'weights': [0.5455969310290191, 0.27389329155169523, 0.7867232615400346], 'output': 0.91979330532386, 'delta': 0.012350248941685238}, {'weights': [0.22216862855566874, -0.12257016233804553, -0.09932831853220148], 'output': -0.0

In [3]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [4]:
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[0], prediction))

Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


In [ ]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row[0].split())
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# load and prepare data
train_filename = 'Digits/ZipDigits.train'
test_filename = 'Digits/ZipDigits.test'
train_dataset = load_csv(train_filename)
for i in range(1, len(train_dataset[0])):
    str_column_to_float(train_dataset, i)
# # convert class column to integers
str_column_to_int(train_dataset, 0)

test_dataset = load_csv(test_filename)
for i in range(1, len(test_dataset[0])):
    str_column_to_float(test_dataset, i)
# # convert class column to integers
str_column_to_int(test_dataset, 0)

{'4.0000': 0,
 '3.0000': 1,
 '5.0000': 2,
 '1.0000': 3,
 '6.0000': 4,
 '0.0000': 5,
 '9.0000': 6,
 '2.0000': 7,
 '7.0000': 8,
 '8.0000': 9}

In [ ]:
minmax = dataset_minmax(dataset)

In [ ]:
normalize_dataset(dataset, minmax)

In [ ]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [ ]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(train_dataset, test_dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [ ]:
n_inputs = len(train_dataset[0]) - 1
n_outputs = len(set([row[0] for row in train_dataset]))
print(n_inputs)
print(n_outputs)
print(len(train_dataset))
network = initialize_network(n_inputs, 100, n_outputs)
train_network(network, train_dataset, 0.5, 10, n_outputs)
for layer in network:
	print(layer)

256
10
7291
>epoch=0, lrate=0.500, error=90515.468
>epoch=1, lrate=0.500, error=76364.103
>epoch=2, lrate=0.500, error=73472.026
>epoch=3, lrate=0.500, error=70799.412
